<h1><center>Lab 6 - A6</center></h1>
<h1><center>Evolutionary Algorithms (2)</center></h1>

In [4]:
import math
import random
import time
import numpy as np
import matplotlib.pyplot as plt

# Sphere Function
The Sphere Function, also called De Jong’s function 1, has the following formula:
$$ f_1(x) = \sum_{i=1}^n x_i^2  \quad -5.12 \leq x_i \leq 5.12 $$
Global optimum: $f(x)=0, \,\, x(i)=0, \,\, i=1:n$.  
It is continuos, convex and unimodal. Number of local minima: no local minimum except the global one.

In [20]:
# Generation and evaluation functions

def sphere_fitness(vector):
  '''
    Returns the sphere function value for the vector x .
    Input:
      - vector: list[float] -> size = n
    Output:
      - value: float
  '''
  sum = 0
  for i in range(len(vector)):
    sum += vector[i] ** 2
  return sum

def generate_vector(n, min_value, max_value):
  '''
    Generates a random vector of real numbers belonging to the interval [min_value, max_value]
    Input:
      - n: int
      - min_value, max_value: float
    Output:
      - vector: list[float]
  '''
  vector = np.random.uniform(low=min_value, high=max_value, size=(n,))
  return vector

In [7]:
# Testing
v = generate_vector(5, -5.12, 5.12)
sphere_fitness(v)

np.float64(47.296306323167)

# Evolutionary Algorithm for the Sphere Function Minimization Problem

In [19]:
# Selection functions

def s_elitism_selection(m, n, population):
  '''
    Returns the best m vectors from the population to be included automatically in the next generation.
    Input:
      - m: int
      - n: int
      - populaton: list[list[float]]
    Output:
      - best_vectors: list[list[float]]
  '''
  population_copy = population[:]
  best_vectors = []
  while len(best_vectors) < m:
    best_fitness = float("inf")
    best_index = None
    for index in range(len(population_copy)):
      vector = population_copy[index]
      fitness = sphere_fitness(vector)
      if fitness < best_fitness:
        best_fitness = fitness
        best_index = index
    best_vector = population_copy[best_index]
    best_vectors.append(best_vector)
    population_copy.pop(best_index)
  return best_vectors

def s_tournament_selection(n, population, k=3):
  '''
    Selects a vector from a given population using tournament selection.
    Input:
      - n: int
      - populaton: list[list[float]]
    Output:
      - parent: list[float]
  '''
  # choose 3 vectors at random and pick the best one
  selected = random.sample(population, k)
  selected.sort(key=lambda vector: sphere_fitness(vector))
  return selected[0]

def find_slice(weights, value):
  '''
    Given a list which represents the roulette wheel slices and their sizes, returns the index of the slice to which the value belongs.
    Input:
      - weights: list[float] -> values in the interval 0 to 1
      - value: float -> in the interval 0 to 1
    Output:
      - index: int
  '''
  # normalize weights to generate distribution
  total_sum = np.sum(weights)
  normalized_weights = [(weight / total_sum) for weight in weights]
  # calculate the cumulative mass function for the discrete distribution
  cumulative  = 0
  cmf = []
  for weight in normalized_weights:
    cumulative += weight
    cmf.append(cumulative )
  # find the index of the slice to which the value belongs
  for index in range(len(cmf)):
    if (cmf[index] > value):
      return index
  return index

def s_proportional_selection(n, population):
  '''
    Selects a vector from a given population using proportional selection.
    The proportion is given by the inverse fitness of each vector.
    Input:
      - n: int
      - populaton: list[list[float]]
    Output:
      - parent: list[float]
  '''
  # configure the roulette wheel slices
  fitnesses = [sphere_fitness(vector) for vector in population]
  # the fitnesses are inverted to ensure that lower fitnesses are more highly valued
  weights = [1 / fitness if fitness != 0 else 1e-6 for fitness in fitnesses]
  # choose a random number and find its corresponding slices
  value = np.random.rand()
  index = find_slice(weights, value)
  # return the vector
  parent = population[index]
  return parent

def s_get_best_vector(population):
  '''
    Returns the best vector and best fitness from a list of vectors
    Input:
      - population: list[list[float]]
    Output:
      - best_vector: list[float]
      - best_fitness: float
  '''
  best_fitness = float("inf")
  best_vector = None
  for curr_vector in population:
    curr_fitness = sphere_fitness(curr_vector)
    if curr_fitness < best_fitness:
      best_fitness = curr_fitness
      best_vector = curr_vector
  return best_vector, best_fitness


In [21]:
# Variation functions

def arithmetic_crossover(vector1, vector2, alpha=None):
  '''
    Recombines the two vectors using the arithmetic crossover strategy.
    Input:
      - vector1, vector2: [list[float]
    Output:
      - child: [list[float]
  '''
  if alpha is None:
    alpha = np.random.rand()
  return alpha * vector1 + (1 - alpha) * vector2

def uniform_crossover(vector1, vector2):
  '''
    Recombines the two vectors using the uniform crossover strategy.
    Input:
      - vector1, vector2: [list[float]]
    Output:
      - child: [list[float]]
  '''
  child = [np.random.choice([value1, value2]) for value1, value2 in zip(vector1, vector2)]
  return child

def gaussian_mutation(vector, min_value, max_value, sigma=0.1):
  '''
    Mutates a vector using the gaussian mutation strategy.
    Input:
      - vector: [list[float]
      - min_value: float
      - max_value: float
    Output:
      - mutated_vector: [list[float]
  '''
  mutated_vector = vector + np.random.normal(0, sigma, size=len(vector))
  mutated_vector = np.clip(mutated_vector, min_value, max_value)
  return mutated_vector

def non_uniform_mutation(vector, iteration, max_iterations, min_value, max_value):
  '''
    Mutates a vector using the non-uniform mutation strategy.
    The mutation step size decreases as the iterations increase.
    Input:
      - vector: [list[float]]
      - iteration: int 
      - max_iterations: int
      - min_value: float
      - max_value: float
    Output:
      - mutated_vector: [list[float]]
  '''
  mutation_factor = 1 - (iteration / max_iterations)
  mutated_vector = []
  for value in vector:
    delta = (max_value - min_value) * mutation_factor * np.random.uniform(-1, 1)
    new_value = value + delta
    new_value = min(max(new_value, min_value), max_value)
    mutated_vector.append(new_value)
  return np.array(mutated_vector)

In [11]:
# Testing
v1 = generate_vector(5, -5.12, 5.12)
v2 = generate_vector(5, -5.12, 5.12)
v3 = generate_vector(5, -5.12, 5.12)
v4 = generate_vector(5, -5.12, 5.12)
population = [v1, v2, v3, v4]
for vector in population:
  print(vector)
print("\nFitnesses:")
print(sphere_fitness(v1), sphere_fitness(v2), sphere_fitness(v3), sphere_fitness(v4))
print("\nElitism selection of 2:")
print(s_elitism_selection(2, 5, population))
print("\nCrossover between first 2 vectors:")
print(arithmetic_crossover(v1, v2))
print("\nMutation of first vector:")
print(gaussian_mutation(v1, -5.12, 5.12))

[-3.77308722 -4.67334564 -0.29080052 -3.63591561 -5.03948277]
[-3.88735661  1.05679011 -3.80439448  2.12936841  2.69718363]
[ 2.94578943 -0.46060318 -1.13047583  4.44782562  3.65223422]
[ 3.19978762 -2.94359136  0.15263124 -5.10991164 -4.59652155]

Fitnesses:
74.77718045578192 42.51077350107185 43.289773822278235 66.1658745740721

Elitism selection of 2:
[array([-3.88735661,  1.05679011, -3.80439448,  2.12936841,  2.69718363]), array([ 2.94578943, -0.46060318, -1.13047583,  4.44782562,  3.65223422])]

Crossover between first 2 vectors:
[-3.83919652 -1.35823882 -2.32355154 -0.30047414 -0.56351978]

Mutation of first vector:
[-3.74343368 -4.40203347 -0.29495222 -3.60538477 -5.12      ]


In [22]:
# Evolutionary algorithm for the Sphere Function minimization problem

def sphere_evolutionary_algorithm(
    n, min_value, max_value,
    population_size=50, elitism_size=None,
    generations=100, selection_strategy='tournament',
    crossover_rate=0.85, crossover_strategy='arithmetic',
    mutation_rate=0.2, mutation_strategy='gaussian'
  ):
  '''
    Implements an evolutionary algorithm for the Sphere Function minimization problem
    Input:
      - n: int
      - min_value, max_value: float
      - population_size: int
      - generations: int
      - elitism_size: int
      - crossover_rate: float -> in the interval 0 to 1
      - mutation_rate: float -> in the interval 0 to 1
      - selection_strategy: {'tournament', 'proportional'}
      - crossover_strategy: {'arithmetic', 'uniform'}
      - mutation_strategy: {'gaussian', 'non-uniform'}
    Output:
      - best_vector: list[int]
      - best_fitness: int
  '''
  # initialize elitism_size to 5% of population
  if elitism_size is None:
    elitism_size = max(1, int(0.1 * population_size))
  
  # initialize population (first generation)
  population = []
  while len(population) < population_size:
    vector = generate_vector(n, min_value, max_value)
    population.append(vector)
  iteration = 0

  # compute the next generations
  while iteration < generations:
    # perform elitism selection
    next_generation = s_elitism_selection(elitism_size, n, population)

    # fill the new generation with children
    while len(next_generation) < population_size:
      # select parents
      if selection_strategy == 'tournament':
        parent1 = s_tournament_selection(n, population)
        parent2 = s_tournament_selection(n, population)
      else:
        parent1 = s_proportional_selection(n, population)
        parent2 = s_proportional_selection(n, population)
      
      if np.random.rand() < crossover_rate:
        # recombine parents to create child
        if crossover_strategy == 'arithmetic':
          child = arithmetic_crossover(parent1, parent2)
        else:
          child = uniform_crossover(parent1, parent2)
      else:
        # child is one of the parents
        child = parent1 if np.random.rand() < 0.5 else parent2
      
      if np.random.rand() < mutation_rate:
        # mutate child
        if mutation_strategy == 'gaussian':
          child = gaussian_mutation(child, min_value, max_value)
        else:
          child = non_uniform_mutation(child, iteration, generations, min_value, max_value)
      
      # add child to next generation
      next_generation.append(child)
    
    # select new population
    population = next_generation
    iteration += 1

  # return best permutation from the final generation
  best_vector, best_fitness = s_get_best_vector(population)
  return best_vector, best_fitness

In [24]:
# Testing the Evolutionary algorithm for the Sphere Function minimization problem

sphere_evolutionary_algorithm(10, -5.12, 5.12, population_size=50, generations=100, elitism_size=10, crossover_rate=0.85, mutation_rate=0.2)

(array([ 0.022751  ,  0.02389183,  0.01016138, -0.02401455, -0.01709305,
         0.04985778, -0.02693316, -0.005192  ,  0.0390402 , -0.04753781]),
 np.float64(0.00908268268378409))

In [25]:
# Function to evaluate the Evolutionary algorithm multiple times

def sphere_evolutionary_algorithm_n_times (
    m, n, min_value, max_value,
    population_size=50, elitism_size=None,
    generations=100, selection_strategy='tournament',
    crossover_rate=0.85, crossover_strategy='arithmetic',
    mutation_rate=0.2, mutation_strategy='gaussian'
):
  '''
    Runs the evolutionary algorithm for the Sphere Function minimization problem m times and returns the best solutions found
    Input:
      - m: int
      - n: int
      - min_value, max_value: float
      - population_size: int
      - generations: int
      - elitism_size: int
      - crossover_rate: float -> in the interval 0 to 1
      - mutation_rate: float -> in the interval 0 to 1
      - selection_strategy: {'tournament', 'proportional'}
      - crossover_strategy: {'arithmetic', 'uniform'}
      - mutation_strategy: {'gaussian', 'non-uniform'}
    Output:
      - best_solutions: list[float]
  '''
  best_solutions = []
  for step in range(m):
    _, best_evaluated = sphere_evolutionary_algorithm(n, min_value, max_value, population_size=population_size, elitism_size=elitism_size, generations=generations, selection_strategy=selection_strategy, crossover_rate=crossover_rate, crossover_strategy=crossover_strategy, mutation_rate=mutation_rate, mutation_strategy=mutation_strategy)
    best_solutions.append(best_evaluated)
  return best_solutions

# Weighted Sphere Model
The Weighted Sphere Model (Axis parallel hyper-ellipsoid) is described by the following formula:
$$ f_1(x) = \sum_{i=1}^n i \cdot x_i^2  \quad -5.12 \leq x_i \leq 5.12 $$
Global optimum: $f(x)=0, \,\, x(i)=0, \,\, i=1:n$.  
It is continuos, convex and unimodal. Number of local minima: no local minimum except the global one.

In [8]:
# Generation and evaluation functions

def weighted_sphere_fitness(vector):
  '''
    Returns the weighted sphere function value for the vector x .
    Input:
      - vector: list[float] -> size = n
    Output:
      - value: float
  '''
  sum = 0
  for i in range(len(vector)):
    sum += i * vector[i] ** 2
  return sum

def generate_vector(n, min_value, max_value):
  '''
    Generates a random vector of real numbers belonging to the interval [min_value, max_value]
    Input:
      - n: int
      - min_value, max_value: float
    Output:
      - vector: list[float]
  '''
  vector = np.random.uniform(low=min_value, high=max_value, size=(n,))
  return vector

# Evolutionary Algorithm for the Weighted Sphere Function Minimization Problem

## Helper functions

In [1]:
# Selection functions

def ws_elitism_selection(m, n, population):
  '''
    Returns the best m vectors from the population to be included automatically in the next generation.
    Input:
      - m: int
      - n: int
      - populaton: list[list[float]]
    Output:
      - best_vectors: list[list[float]]
  '''
  population_copy = population[:]
  best_vectors = []
  while len(best_vectors) < m:
    best_fitness = float("inf")
    best_index = None
    for index in range(len(population_copy)):
      vector = population_copy[index]
      fitness = weighted_sphere_fitness(vector)
      if fitness < best_fitness:
        best_fitness = fitness
        best_index = index
    best_vector = population_copy[best_index]
    best_vectors.append(best_vector)
    population_copy.pop(best_index)
  return best_vectors

def ws_tournament_selection(n, population, k=3):
  '''
    Selects a vector from a given population using tournament selection.
    Input:
      - n: int
      - populaton: list[list[float]]
    Output:
      - parent: list[float]
  '''
  # choose 3 vectors at random and pick the best one
  selected = random.sample(population, k)
  selected.sort(key=lambda vector: weighted_sphere_fitness(vector))
  return selected[0]

def find_slice(weights, value):
  '''
    Given a list which represents the roulette wheel slices and their sizes, returns the index of the slice to which the value belongs.
    Input:
      - weights: list[float] -> values in the interval 0 to 1
      - value: float -> in the interval 0 to 1
    Output:
      - index: int
  '''
  # normalize weights to generate distribution
  total_sum = np.sum(weights)
  normalized_weights = [(weight / total_sum) for weight in weights]
  # calculate the cumulative mass function for the discrete distribution
  cumulative  = 0
  cmf = []
  for weight in normalized_weights:
    cumulative += weight
    cmf.append(cumulative )
  # find the index of the slice to which the value belongs
  for index in range(len(cmf)):
    if (cmf[index] > value):
      return index
  return index

def ws_proportional_selection(n, population):
  '''
    Selects a vector from a given population using proportional selection.
    The proportion is given by the inverse fitness of each vector.
    Input:
      - n: int
      - populaton: list[list[float]]
    Output:
      - parent: list[float]
  '''
  # configure the roulette wheel slices
  fitnesses = [weighted_sphere_fitness(vector) for vector in population]
  # the fitnesses are inverted to ensure that lower fitnesses are more highly valued
  weights = [1 / fitness if fitness != 0 else 1e-6 for fitness in fitnesses]
  # choose a random number and find its corresponding slices
  value = np.random.rand()
  index = find_slice(weights, value)
  # return the vector
  parent = population[index]
  return parent

def ws_get_best_vector(population):
  '''
    Returns the best vector and best fitness from a list of vectors
    Input:
      - population: list[list[float]]
    Output:
      - best_vector: list[float]
      - best_fitness: float
  '''
  best_fitness = float("inf")
  best_vector = None
  for curr_vector in population:
    curr_fitness = weighted_sphere_fitness(curr_vector)
    if curr_fitness < best_fitness:
      best_fitness = curr_fitness
      best_vector = curr_vector
  return best_vector, best_fitness

In [89]:
# Variation functions

def arithmetic_crossover(vector1, vector2, alpha=None):
  '''
    Recombines the two vectors using the arithmetic crossover strategy.
    Input:
      - vector1, vector2: [list[float]
    Output:
      - child: [list[float]
  '''
  if alpha is None:
    alpha = np.random.rand()
  return alpha * vector1 + (1 - alpha) * vector2

def uniform_crossover(vector1, vector2):
  '''
    Recombines the two vectors using the uniform crossover strategy.
    Input:
      - vector1, vector2: [list[float]]
    Output:
      - child: [list[float]]
  '''
  child = [np.random.choice([value1, value2]) for value1, value2 in zip(vector1, vector2)]
  return child

def gaussian_mutation(vector, min_value, max_value, sigma=0.1):
  '''
    Mutates a vector using the gaussian mutation strategy.
    Input:
      - vector: [list[float]
      - min_value: float
      - max_value: float
    Output:
      - mutated_vector: [list[float]
  '''
  mutated_vector = vector + np.random.normal(0, sigma, size=len(vector))
  mutated_vector = np.clip(mutated_vector, min_value, max_value)
  return mutated_vector

def non_uniform_mutation(vector, iteration, max_iterations, min_value, max_value):
  '''
    Mutates a vector using the non-uniform mutation strategy.
    The mutation step size decreases as the iterations increase.
    Input:
      - vector: [list[float]]
      - iteration: int 
      - max_iterations: int
      - min_value: float
      - max_value: float
    Output:
      - mutated_vector: [list[float]]
  '''
  mutation_factor = 1 - (iteration / max_iterations)
  mutated_vector = []
  for value in vector:
    delta = (max_value - min_value) * mutation_factor * np.random.uniform(-1, 1)
    new_value = value + delta
    new_value = min(max(new_value, min_value), max_value)
    mutated_vector.append(new_value)
  return np.array(mutated_vector)

## Implementation

In [76]:
# Evolutionary algorithm for the Weighted Sphere Function minimization problem

def weighted_sphere_evolutionary_algorithm(
    n, min_value, max_value,
    population_size=50, elitism_size=None,
    generations=100, selection_strategy='tournament',
    crossover_rate=0.85, crossover_strategy='arithmetic',
    mutation_rate=0.2, mutation_strategy='gaussian'
  ):
  '''
    Implements an evolutionary algorithm for the Weighted Sphere Function minimization problem
    Input:
      - n: int
      - min_value, max_value: float
      - population_size: int
      - generations: int
      - elitism_size: int
      - crossover_rate: float -> in the interval 0 to 1
      - mutation_rate: float -> in the interval 0 to 1
      - selection_strategy: {'tournament', 'proportional'}
      - crossover_strategy: {'arithmetic', 'uniform'}
      - mutation_strategy: {'gaussian', 'non-uniform'}
    Output:
      - best_vector: list[int]
      - best_fitness: int
  '''
  # initialize elitism_size to 5% of population
  if elitism_size is None:
    elitism_size = max(1, int(0.1 * population_size))
  
  # initialize population (first generation)
  population = []
  while len(population) < population_size:
    vector = generate_vector(n, min_value, max_value)
    population.append(vector)
  iteration = 0

  # compute the next generations
  while iteration < generations:
    # perform elitism selection
    next_generation = ws_elitism_selection(elitism_size, n, population)

    # fill the new generation with children
    while len(next_generation) < population_size:
      # select parents
      if selection_strategy == 'tournament':
        parent1 = ws_tournament_selection(n, population)
        parent2 = ws_tournament_selection(n, population)
      else:
        parent1 = ws_proportional_selection(n, population)
        parent2 = ws_proportional_selection(n, population)
      
      if np.random.rand() < crossover_rate:
        # recombine parents to create child
        if crossover_strategy == 'arithmetic':
          child = arithmetic_crossover(parent1, parent2)
        else:
          child = uniform_crossover(parent1, parent2)
      else:
        # child is one of the parents
        child = parent1 if np.random.rand() < 0.5 else parent2
      
      if np.random.rand() < mutation_rate:
        # mutate child
        if mutation_strategy == 'gaussian':
          child = gaussian_mutation(child, min_value, max_value)
        else:
          child = non_uniform_mutation(child, iteration, generations, min_value, max_value)
      
      # add child to next generation
      next_generation.append(child)
    
    # select new population
    population = next_generation
    iteration += 1

  # return best permutation from the final generation
  best_vector, best_fitness = ws_get_best_vector(population)
  return best_vector, best_fitness

In [57]:
# Testing the Evolutionary algorithm for the Weighted Sphere Function minimization problem

weighted_sphere_evolutionary_algorithm(10, -5.12, 5.12, population_size=50, generations=100, elitism_size=10, crossover_rate=0.85, mutation_rate=0.2)

(array([ 0.10346133, -0.02077034, -0.01191243, -0.04562593, -0.03195237,
        -0.00398574,  0.02404109, -0.00158717,  0.02454968, -0.01870223]),
 np.float64(0.022578576913212148))

In [58]:
# Function to evaluate the Evolutionary algorithm multiple times

def weighted_sphere_evolutionary_algorithm_n_times (
    m, n, min_value, max_value,
    population_size=50, elitism_size=None,
    generations=100, selection_strategy='tournament',
    crossover_rate=0.85, crossover_strategy='arithmetic',
    mutation_rate=0.2, mutation_strategy='gaussian'
):
  '''
    Runs the evolutionary algorithm for the Weighted Sphere Function minimization problem m times and returns the best solutions found
    Input:
      - m: int
      - n: int
      - min_value, max_value: float
      - population_size: int
      - generations: int
      - elitism_size: int
      - crossover_rate: float -> in the interval 0 to 1
      - mutation_rate: float -> in the interval 0 to 1
      - selection_strategy: {'tournament', 'proportional'}
      - crossover_strategy: {'arithmetic', 'uniform'}
      - mutation_strategy: {'gaussian', 'non-uniform'}
    Output:
      - best_solutions: list[float]
  '''
  best_solutions = []
  for step in range(m):
    _, best_evaluated = weighted_sphere_evolutionary_algorithm(n, min_value, max_value, population_size=population_size, elitism_size=elitism_size, generations=generations, selection_strategy=selection_strategy, crossover_rate=crossover_rate, crossover_strategy=crossover_strategy, mutation_rate=mutation_rate, mutation_strategy=mutation_strategy)
    best_solutions.append(best_evaluated)
  return best_solutions

## Comparative analysis for Weighted Sphere Minimization Problem

In [28]:
# Generate markdown tables from data

def lists_to_markdown_table(header, *lists):
  '''
    Returns a string formatted like a markdown table which contains data from the header and the lists
    Input:
      - header: string
      - *lists: varying number of list[] -> size = n
    Output:
      - markdown_table: string
  '''
  markdown_table = header
  n = len(lists[0])
  for i in range(n):
    markdown_table += "|"
    for list in lists:
      markdown_table += f" {list[i]} |"
    markdown_table += "\n"
  return markdown_table

In [68]:
# Test the algorithm for different values of n and generations

# number of dimensions
n_values = [10, 50, 100]
generations_values = [100, 500, 1000]
min_value, max_value = -5.12, 5.12

# total expermients: 3 * 3 = 9

n_list = []
generations_list = []
fitness_list = []
execution_time_list = []

for n in n_values:
  for generations in generations_values:
    # perform 5 experiments for given values
    start_time = time.time()
    best_fitnesses = weighted_sphere_evolutionary_algorithm_n_times(5, n, min_value, max_value, generations=generations)
    end_time = time.time()

    average_fitness = np.array(best_fitnesses).mean()
    average_execution_time = (end_time - start_time) / 5

    n_list.append(n)
    generations_list.append(generations)
    fitness_list.append(average_fitness)
    execution_time_list.append(average_execution_time)

In [71]:
# display experiment results in markdown table
header = "| Vector dimension | Generations number | Fitness | Execution time |\n"
header += "|---|---|---|---|\n"
markdown_table = lists_to_markdown_table(header, n_list, generations_list, fitness_list, execution_time_list)

| Vector dimension | Generations number | Fitness | Execution time |
|---|---|---|---|
| 10 | 100 | 0.07945685425917352 | 0.17840447425842285 |
| 10 | 500 | 0.006698671811744168 | 0.907810640335083 |
| 10 | 1000 | 0.003665321327124712 | 1.8022840023040771 |
| 50 | 100 | 47.432079543666745 | 0.6791293621063232 |
| 50 | 500 | 14.572759220349903 | 3.3594626903533937 |
| 50 | 1000 | 7.61788838295341 | 7.038908863067627 |
| 100 | 100 | 580.2210811779107 | 1.3718093395233155 |
| 100 | 500 | 125.57737102523129 | 7.212801551818847 |
| 100 | 1000 | 79.8179981684506 | 12.525879621505737 |

### Experiment findings:
- By increasing the number of dimensions considered, the overall quality of the solutions in terms of fitness and execution time decreases, as to be expected.
- This is to be compensated with increasing the depth of execution (in terms of Generations number).
- We see that, generally, the algorithm is more performant for higher values of generations, and when the dimensionality is also large this plays a huge role in the overall performance of the algorithm.
  
For further experiments, in order to analyze a relevant test case, while also preserving computational power, I will choose the configuration ```n = 50, generations = 500```.

In [92]:
# Test the algorithm for different types of selection_strategy, crossover_strategy and mutation_strategy

selection_strategy_options = ['tournament', 'proportional']
crossover_strategy_options = ['arithmetic', 'uniform']
mutation_strategy_options = ['gaussian', 'non-uniform']

# total expermients: 2 * 2 * 2 = 8

selection_strategy_list = []
crossover_strategy_list = []
mutation_strategy_list = []

fitness_list = []
execution_time_list = []

for selection_strategy in selection_strategy_options:
  for crossover_strategy in crossover_strategy_options:
    for mutation_strategy in mutation_strategy_options:
      # perform 5 experiments for given values
      start_time = time.time()
      best_fitnesses = weighted_sphere_evolutionary_algorithm_n_times(5, 50, min_value, max_value, generations=500, selection_strategy=selection_strategy, crossover_strategy=crossover_strategy, mutation_strategy=mutation_strategy)
      end_time = time.time()

      average_fitness = np.array(best_fitnesses).mean()
      average_execution_time = (end_time - start_time) / 5

      selection_strategy_list.append(selection_strategy)
      crossover_strategy_list.append(crossover_strategy)
      mutation_strategy_list.append(mutation_strategy)
      
      fitness_list.append(average_fitness)
      execution_time_list.append(average_execution_time)

In [94]:
# display experiment results in markdown table
header = "| Selection strategy | Crossover strategy | Mutation strategy | Fitness | Execution time |\n"
header += "|---|---|---|---|---|\n"
markdown_table = lists_to_markdown_table(header, selection_strategy_list, crossover_strategy_list, mutation_strategy_list, fitness_list, execution_time_list)

| Selection strategy | Crossover strategy | Mutation strategy | Fitness | Execution time |
|---|---|---|---|---|
| tournament | arithmetic | gaussian | 11.180907808247571 | 3.2948512554168703 |
| tournament | arithmetic | non-uniform | 139.64645439875355 | 4.042527103424073 |
| tournament | uniform | gaussian | 18.349591884653158 | 11.559218168258667 |
| tournament | uniform | non-uniform | 161.29733634554253 | 11.529843711853028 |
| proportional | arithmetic | gaussian | 5.183882369928136 | 29.144227600097658 |
| proportional | arithmetic | non-uniform | 95.69582703891027 | 35.40359144210815 |
| proportional | uniform | gaussian | 9.351347620410053 | 36.22637157440185 |
| proportional | uniform | non-uniform | 75.53841660112747 | 34.0156662940979 |

In [97]:
# Analyse fitness by each parameter option
selection_strategy_options = ['tournament', 'proportional']
crossover_strategy_options = ['arithmetic', 'uniform']
mutation_strategy_options = ['gaussian', 'non-uniform']

# average fitness and average time by selection_strategy
for selection_strategy in selection_strategy_options:
  fitnesses = []
  execution_times = []
  for i in range(8):
    if selection_strategy_list[i] == selection_strategy:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"Selection strategy: {selection_strategy}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by crossover_strategy
for crossover_strategy in crossover_strategy_options:
  fitnesses = []
  execution_times = []
  for i in range(8):
    if crossover_strategy_list[i] == crossover_strategy:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"Crossover strategy: {crossover_strategy}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by mutation_strategy
for mutation_strategy in mutation_strategy_options:
  fitnesses = []
  execution_times = []
  for i in range(8):
    if mutation_strategy_list[i] == mutation_strategy:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"Mutation strategy: {mutation_strategy}, average fitness: {average_fitness}, average execution time: {average_execution_time}")

Selection strategy: tournament, average fitness: 82.6185726092992, average execution time: 7.606610059738159
Selection strategy: proportional, average fitness: 46.44236840759398, average execution time: 33.69746422767639

Crossover strategy: arithmetic, average fitness: 62.926767903959885, average execution time: 17.97129935026169
Crossover strategy: uniform, average fitness: 66.1341731129333, average execution time: 23.332774937152863

Mutation strategy: gaussian, average fitness: 11.01643242080973, average execution time: 20.05616714954376
Mutation strategy: non-uniform, average fitness: 118.04450859608343, average execution time: 21.247907137870786


### Experiment findings:
When analyzing the impact of different strategies employed, the following have to be considered:  
  
**Selection strategy**:  
- in terms of fitness, proportional strategy is significantly more performant, while it is vastly more time consuming
- for the most accurate results, proportional strategy should be employed
- for faster, decent results, tournament trategy performs well
  
**Crossover strategy**:
- the difference between the two strategies in terms of performance is not very significant
- for slightly better results overall, arithmetic strategy is preffered
  
**Mutation strategy**:
- the gaussian strategy vastly outperforms the non-uniform selection strategy in terms of fitness, while execution time does not vary much between the two
- this is a sign that a proportional approach to exploration, based on the number of current iterations is not well fit for this algorithm, since it makes very slow progress in exploring the search space and does not reach a good value in reasonable time
  
Taking these into consideration, we can assure that the best setup for a run of the evolutionary algorithm for the weighted sphere problem is by employing ```tournament selection, arithmetic crossover, gaussian mutation```. This ensures good solution quality within reasonable execution time.  
For further optimization, the best improvement is using ```proportional (roulette wheel) selection``` to ensure the best fitness results (albeit at a higher computational cost).

## Comparative analysis for Sphere Minimization Problem

In [26]:
# Test the algorithm for different types of selection_strategy, crossover_strategy and mutation_strategy

min_value = -5.12
max_value = 5.12

selection_strategy_options = ['tournament', 'proportional']
crossover_strategy_options = ['arithmetic', 'uniform']
mutation_strategy_options = ['gaussian', 'non-uniform']

# total expermients: 2 * 2 * 2 = 8

selection_strategy_list = []
crossover_strategy_list = []
mutation_strategy_list = []

fitness_list = []
execution_time_list = []

for selection_strategy in selection_strategy_options:
  for crossover_strategy in crossover_strategy_options:
    for mutation_strategy in mutation_strategy_options:
      # perform 5 experiments for given values
      start_time = time.time()
      best_fitnesses = sphere_evolutionary_algorithm_n_times(5, 10, min_value, max_value, generations=100, selection_strategy=selection_strategy, crossover_strategy=crossover_strategy, mutation_strategy=mutation_strategy)
      end_time = time.time()

      average_fitness = np.array(best_fitnesses).mean()
      average_execution_time = (end_time - start_time) / 5

      selection_strategy_list.append(selection_strategy)
      crossover_strategy_list.append(crossover_strategy)
      mutation_strategy_list.append(mutation_strategy)
      
      fitness_list.append(average_fitness)
      execution_time_list.append(average_execution_time)

In [29]:
# display experiment results in markdown table
header = "| Selection strategy | Crossover strategy | Mutation strategy | Fitness | Execution time |\n"
header += "|---|---|---|---|---|\n"
markdown_table = lists_to_markdown_table(header, selection_strategy_list, crossover_strategy_list, mutation_strategy_list, fitness_list, execution_time_list)

| Selection strategy | Crossover strategy | Mutation strategy | Fitness | Execution time |
|---|---|---|---|---|
| tournament | arithmetic | gaussian | 0.0063779341569195165 | 0.15794901847839354 |
| tournament | arithmetic | non-uniform | 0.540419538179014 | 0.17046995162963868 |
| tournament | uniform | gaussian | 0.010899804079687794 | 0.4457678318023682 |
| tournament | uniform | non-uniform | 0.453418820637259 | 0.47041988372802734 |
| proportional | arithmetic | gaussian | 0.003675112585362128 | 1.1570385456085206 |
| proportional | arithmetic | non-uniform | 0.23108554711151888 | 1.1931095600128174 |
| proportional | uniform | gaussian | 0.004928819660180241 | 1.2839437007904053 |
| proportional | uniform | non-uniform | 0.24357164242406518 | 1.3218822479248047 |


In [31]:
# Analyse fitness by each parameter option
selection_strategy_options = ['tournament', 'proportional']
crossover_strategy_options = ['arithmetic', 'uniform']
mutation_strategy_options = ['gaussian', 'non-uniform']

# average fitness and average time by selection_strategy
for selection_strategy in selection_strategy_options:
  fitnesses = []
  execution_times = []
  for i in range(8):
    if selection_strategy_list[i] == selection_strategy:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"Selection strategy: {selection_strategy}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by crossover_strategy
for crossover_strategy in crossover_strategy_options:
  fitnesses = []
  execution_times = []
  for i in range(8):
    if crossover_strategy_list[i] == crossover_strategy:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"Crossover strategy: {crossover_strategy}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by mutation_strategy
for mutation_strategy in mutation_strategy_options:
  fitnesses = []
  execution_times = []
  for i in range(8):
    if mutation_strategy_list[i] == mutation_strategy:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"Mutation strategy: {mutation_strategy}, average fitness: {average_fitness}, average execution time: {average_execution_time}")

Selection strategy: tournament, average fitness: 0.2527790242632201, average execution time: 0.3111516714096069
Selection strategy: proportional, average fitness: 0.12081528044528161, average execution time: 1.238993513584137

Crossover strategy: arithmetic, average fitness: 0.19538953300820364, average execution time: 0.6696417689323426
Crossover strategy: uniform, average fitness: 0.17820477170029805, average execution time: 0.8805034160614014

Mutation strategy: gaussian, average fitness: 0.00647041762053742, average execution time: 0.761174774169922
Mutation strategy: non-uniform, average fitness: 0.3671238870879643, average execution time: 0.788970410823822


### Experiment findings:
Although this time I chose a smaller n and generations time (for time saving purposes), we see very similar results when comparing the performance of the algorithm for the Sphere Function to that for the Weighted Sphere Function.
- The best setup for a run of the evolutionary algorithm for the sphere problem is by employing ```tournament selection, arithmetic crossover, gaussian mutation```. This ensures good solution quality within reasonable execution time.  
  
- For further optimization, the best improvement is using ```proportional (roulette wheel) selection``` to ensure the best fitness results (albeit at a higher computational cost).